# Making McDonald’s Healthy

first importing our packages

In [2]:
!pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install chart_studio

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=fc4c6190c00809bc08ab46ab9258db9a2990c4bf40ad13472e42b5baa8fb2f8e
  Stored in directory: c:\users\aksha\appdata\local\pip\cache\wheels\ce\18\7f\e9527e3e66db1456194ac7f61eb3211068c409edceecff2d31
Successfully built retrying



In [7]:
import numpy as np
import pandas as pd
from pulp import *
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import matplotlib.pyplot as plt # matplotlib
import os

In [9]:
McData = pd.read_csv('menu.csv')

Let’s look at a plot of the Carbohydrates vs. Calories coloured by menu item type. 

In [10]:
def make_scatter(McData,category,x_cat,y_cat):
    return  go.Scatter(
       x = McData[McData['Category'].isin([category])][x_cat],
       y = McData[McData['Category'].isin([category])][y_cat],
       mode = "markers",
       name = category,
       text=  McData.Item)

In [11]:
# Define our categories to plot
x_cat = 'Calories'; y_cat = 'Carbohydrates'
# Create a list of scatter plots to view all at once
data = [make_scatter(McData,cat,x_cat,y_cat) for cat in
   McData.Category.unique().tolist()]
# Define the plot layout (title, ticks etc.)
layout = dict(title = 'McDonalds Nutrition',
   xaxis= dict(title= 'Calories',ticklen=5,zeroline= False),
   yaxis= dict(title= 'Carbohydrates(g)',ticklen= 5,zeroline=False))
# Finally we will plot the data with the layout
fig = dict(data = data, layout = layout)
iplot(fig)


Let us do another one

In [13]:
# Convert the item names to a list
MenuItems = McData.Item.tolist()
# Convert all of the macro nutrients fields to be dictionaries of the item names
Calories = McData.set_index('Item')['Calories'].to_dict()
TotalFat = McData.set_index('Item')['Total Fat'].to_dict()
SaturatedFat = McData.set_index('Item')['Saturated Fat'].to_dict()
Carbohydrates = McData.set_index('Item')['Carbohydrates'].to_dict()
Sugars = McData.set_index('Item')['Sugars'].to_dict()
Protein = McData.set_index('Item')['Protein'].to_dict()
Sodium = McData.set_index('Item')['Sodium'].to_dict()

In [19]:
# Set it up as a minimization problem
prob = LpProblem("McOptimizationProblem", LpMinimize)

In [15]:
MenuItems_vars = LpVariable.dicts("MenuItems",MenuItems,lowBound=0,
   upBound=10,cat='Integer')

In [21]:
# First entry is the calorie calculation (this is our objective)
prob += lpSum([Calories[i]*MenuItems_vars[i] for i in MenuItems]),"Calories"
# Total Fat must be <= 70 g
prob += lpSum([TotalFat[i]*MenuItems_vars[i] for i in MenuItems]) <=70, "TotalFat"
# Saturated Fat is <= 20 g
prob += lpSum([SaturatedFat[i]*MenuItems_vars[i] for i in MenuItems]) <= 20, "Saturated Fat"
# Carbohydrates must be more than 260 g
prob += lpSum([Carbohydrates[i]*MenuItems_vars[i] for i in MenuItems]) >= 260, "Carbohydrates_lower"
# Sugar between 80-100 g
prob += lpSum([Sugars[i]*MenuItems_vars[i] for i in MenuItems]) >=80, "Sugars_lower"
prob += lpSum([Sugars[i]*MenuItems_vars[i] for i in MenuItems]) <=100, "Sugars_upper"
# Protein between 45-55g
prob += lpSum([Protein[i]*MenuItems_vars[i] for i in MenuItems]) >=45, "Protein_lower"
prob += lpSum([Protein[i]*MenuItems_vars[i] for i in MenuItems]) <=55, "Protein_upper"
# Sodium <= 6000 mg
prob += lpSum([Sodium[i]*MenuItems_vars[i] for i in MenuItems]) <=6000, "Sodium"

In [22]:
# Tadaaaaa
prob.solve()

1

In [23]:
print("Status:", LpStatus[prob.status])

Status: Optimal
